In [40]:
import numpy as np
import pandas as pd
from random import seed
import re
import string
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.util import ngrams
import stop_words
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

In [47]:
df = pd.read_csv('.\Downloads\Engage_V1.0.csv',encoding = 'latin1')

In [48]:
df['desc_clean'] = df['Description'].str.lower()
df['desc_clean'] = df['desc_clean'].apply(lambda row: re.sub('[^a-zA-Z0-9#.:-_]',' ',row))
df['desc_clean'] = df['desc_clean'].apply(lambda row: re.sub('[+>\/<,)¿(&?:?%*=-]+','',row))
df['desc_clean'] = df['desc_clean'].apply(lambda row: re.sub('\s+[.]','',row))
df['desc_clean'] = df['desc_clean'].apply(lambda row: re.sub('\d+','',row))
df['desc_clean'] = df['desc_clean'].apply(lambda row: re.sub('http\S','',row))

In [51]:
wnl =  WordNetLemmatizer()
df['desc_clean']  = df['desc_clean'].apply(lambda row: wnl.lemmatize(row))

In [52]:
df['desc_clean']

0        for amarpatan region of madhya pradesh  we wis...
1        consumer called up and asked about if he can g...
2        consumer called up as he has a pack of nestl  ...
3        retailer called in again and said that his con...
4        consumer called to inquire about the method of...
5        consumer called up as he wanted to know about ...
6        consumer called us and shared the feedback for...
7        consumer called in as she is using high protei...
8        source nestle com   firstname sharad  lastname...
9        customer is looking for a technician for two v...
10       consumer called to inquire about the shelf lif...
11       source nestle com   firstname jay  lastname de...
12       consumer is looking for technician for nns ven...
13       hi gudevening   iam nishan karkera here  at pr...
14       customer called up as he want distributorship ...
15       consumer is looking for a technician as she sa...
16       consumer called saying that he is feeding lact.

In [53]:
df['desc_clean'] = df['desc_clean'].apply(lambda row:  re.split(r' ',row)).apply(lambda row: list(filter(None,row)))

In [56]:
stoplist = nltk.corpus.stopwords.words('english')
combine_list = [word for word in stop_words.get_stop_words('en') if word not in nltk.corpus.stopwords.words('english')]
stoplist.extend(combine_list)

In [58]:
df['desc_clean'] = df['desc_clean'].apply(lambda row: [word for word in row if word not in stoplist])

In [59]:
df['desc_clean']

0        [amarpatan, region, madhya, pradesh, wish, nes...
1        [consumer, called, asked, give, nestogen, mont...
2        [consumer, called, pack, nestl, cerelac, wheat...
3        [retailer, called, said, concern, registered, ...
4        [consumer, called, inquire, method, preparatio...
5        [consumer, called, wanted, know, expiration, d...
6        [consumer, called, us, shared, feedback, nestl...
7        [consumer, called, using, high, protein, resou...
8        [source, nestle, com, firstname, sharad, lastn...
9        [customer, looking, technician, two, vending, ...
10       [consumer, called, inquire, shelf, life, maggi...
11       [source, nestle, com, firstname, jay, lastname...
12       [consumer, looking, technician, nns, vending, ...
13       [hi, gudevening, iam, nishan, karkera, present...
14       [customer, called, want, distributorship, area...
15       [consumer, looking, technician, said, still, n...
16       [consumer, called, saying, feeding, lactogen, .

In [60]:
df['desc_clean'] = df['desc_clean'].apply(' '.join)

In [61]:
bigrm = df['desc_clean'].apply(lambda row: nltk.bigrams(row.split()))

In [62]:
x_train,x_test,y_train,y_test = train_test_split(df['desc_clean'], bigrm, random_state = 42, test_size = 0.3)

In [63]:
cv = CountVectorizer(ngram_range = (1,2))
X_train = cv.fit_transform(df['desc_clean'])
X_test = cv.transform(x_test)

In [64]:
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

In [65]:
#Converting sparse training and testing datasets to dense matrices for classification
X_train = X_train.todense()
X_test = X_test.todense()
#Classification using Multinomial Naive Bayes
classifier = MultinomialNB()
classifier.fit(X_train,y_train)
prediction = classifier.predict(X_test)
accuracy_score(y_true = y_test, y_pred = prediction)

MemoryError: 

(16633, 26)